In [1]:
from pyspark.sql import SparkSession
import apache_access_log

In [2]:
spark = SparkSession.builder.appName("Log Analyzer").getOrCreate()
sc = spark.sparkContext

In [3]:
logFile = "/home/wengong/spark/apps/databrick-ref-apps/logs_analyzer/data/apache.access.log"

In [4]:
access_logs = sc.textFile(logFile).map(apache_access_log.parse_apache_log_line)

In [5]:
type(access_logs)

pyspark.rdd.PipelinedRDD

In [11]:
row1 = access_logs.first()

In [12]:
type(row1)

pyspark.sql.types.Row

In [15]:
columns = list(row1.asDict().keys())

In [16]:
columns

['ip_address',
 'client_identd',
 'user_id',
 'date_time',
 'method',
 'endpoint',
 'protocol',
 'response_code',
 'content_size']

In [17]:
df = access_logs.toDF(columns)

In [20]:
df = df.drop("client_identd", "user_id")

In [21]:
df.show(5)

+------------+--------------------+------+--------------------+--------+-------------+------------+
|  ip_address|           date_time|method|            endpoint|protocol|response_code|content_size|
+------------+--------------------+------+--------------------+--------+-------------+------------+
|64.242.88.10|07/Mar/2004:16:05...|   GET|/twiki/bin/edit/M...|HTTP/1.1|          401|       12846|
|64.242.88.10|07/Mar/2004:16:06...|   GET|/twiki/bin/rdiff/...|HTTP/1.1|          200|        4523|
|64.242.88.10|07/Mar/2004:16:10...|   GET|/mailman/listinfo...|HTTP/1.1|          200|        6291|
|64.242.88.10|07/Mar/2004:16:11...|   GET|/twiki/bin/view/T...|HTTP/1.1|          200|        7352|
|64.242.88.10|07/Mar/2004:16:20...|   GET|/twiki/bin/view/M...|HTTP/1.1|          200|        5253|
+------------+--------------------+------+--------------------+--------+-------------+------------+
only showing top 5 rows



In [22]:
df.createOrReplaceTempView("logs")

In [24]:
# Calculate statistics based on the content size.

In [23]:
sql = """
SELECT 
    SUM(content_size) as theSum,
    COUNT(*) as theCount,
    Avg(content_size) as theAvg,
    MIN(content_size) as theMin,
    MAX(content_size) as theMax
FROM logs
"""
spark.sql(sql).show()

+--------+--------+-----------------+------+------+
|  theSum|theCount|           theAvg|theMin|theMax|
+--------+--------+-----------------+------+------+
|10933005|    1406|7775.963726884779|     0|138789|
+--------+--------+-----------------+------+------+



In [ ]:
# Response Code to Count

In [28]:
sql = """
SELECT response_code, COUNT(*) AS theCount 
FROM logs 
GROUP BY response_code 
order by COUNT(*) desc
"""
spark.sql(sql).show()

+-------------+--------+
|response_code|theCount|
+-------------+--------+
|          200|    1272|
|          401|     123|
|          302|       6|
|          404|       5|
+-------------+--------+



In [29]:
# Any IPAddress that has accessed the server more than 10 times

In [41]:
sql = """
SELECT ip_address, COUNT(*) AS theCount 
FROM logs 
GROUP BY ip_address 
order by COUNT(*) desc
"""
spark.sql(sql).show(10, truncate=False)

+-------------------------------+--------+
|ip_address                     |theCount|
+-------------------------------+--------+
|64.242.88.10                   |452     |
|10.0.0.153                     |188     |
|cr020r01-3.sac.overture.com    |44      |
|h24-71-236-129.ca.shawcable.net|36      |
|h24-70-69-74.ca.shawcable.net  |32      |
|market-mail.panduit.com        |29      |
|ts04-ip92.hevanet.com          |28      |
|ip68-228-43-49.tc.ph.cox.net   |22      |
|proxy0.haifa.ac.il             |19      |
|207.195.59.160                 |15      |
+-------------------------------+--------+
only showing top 10 rows



In [ ]:
# Top Endpoints

In [42]:
sql = """
SELECT endpoint, COUNT(*) AS theCount 
FROM logs 
GROUP BY endpoint 
order by COUNT(*) desc
"""
spark.sql(sql).show(10, truncate=False)

+-----------------------------------------------+--------+
|endpoint                                       |theCount|
+-----------------------------------------------+--------+
|/twiki/bin/view/Main/WebHome                   |40      |
|/twiki/pub/TWiki/TWikiLogos/twikiRobot46x50.gif|32      |
|/                                              |31      |
|/favicon.ico                                   |28      |
|/robots.txt                                    |27      |
|/razor.html                                    |23      |
|/twiki/bin/view/Main/SpamAssassinTaggingOnly   |18      |
|/twiki/bin/view/Main/SpamAssassinAndPostFix    |17      |
|/cgi-bin/mailgraph.cgi/mailgraph_1_err.png     |16      |
|/cgi-bin/mailgraph.cgi/mailgraph_3.png         |16      |
+-----------------------------------------------+--------+
only showing top 10 rows



In [ ]:
spark.stop()